# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [2]:
data = pd.read_csv("https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv",
                  delimiter="~")

In [3]:
data.iloc[[13,27,101]]

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,"Finalwt_worksite,,,,"
13,3,1.0,2.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,1437.0,4.0,6.0,7.0,3.0,0.0,"47.793940929,,,,"
27,1,3.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,1.0,2501.0,2.0,4.0,7.0,8.0,0.0,"47.793940929,,,,"
101,2,1.0,1.0,3.0,2.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,2.0,12636.0,4.0,6.0,7.0,4.0,0.0,"47.793940929,,,,"


## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [4]:
dfcols = ["Industry", "Size", "OC3", "HI1", "HI2", "HI3", "CP1", "WL6"]+list(data.columns[data.columns.str.startswith("WD")])

In [5]:
df = data.loc[:,dfcols]

In [6]:
df.head()

,Industry,Size,OC3,HI1,HI2,HI3,CP1,WL6,WD1_1,WD1_2,WD2,WD3,WD4,WD5,WD6,WD7
0,7.0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,7.0,6.0,3.0,2.0,3.0,1.0,1.0,1.0,997.0,997.0,90.0,90.0,997.0,997.0,0.0,997.0
2,7.0,8.0,3.0,1.0,3.0,1.0,1.0,1.0,35.0,4.0,997.0,997.0,40.0,15.0,997.0,997.0
3,7.0,4.0,2.0,1.0,2.0,1.0,2.0,2.0,50.0,15.0,50.0,85.0,75.0,0.0,0.0,997.0
4,7.0,4.0,3.0,1.0,3.0,1.0,1.0,1.0,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [7]:
["Industry", "Size", "OC3", "HI1", "HI2", "HI3", "CP1", "WL6"]+list(data.columns[data.columns.str.startswith("WD")])

['Industry',
 'Size',
 'OC3',
 'HI1',
 'HI2',
 'HI3',
 'CP1',
 'WL6',
 'WD1_1',
 'WD1_2',
 'WD2',
 'WD3',
 'WD4',
 'WD5',
 'WD6',
 'WD7']

In [8]:
newcols = {
    'Industry': 'Industry',
    'Size': 'Employee_Size',
    'OC3': 'Organization_Type_Profit',
    'HI1': 'Health_Insur_Type',
    'HI2': 'Personal_Health_Insur',
    'HI3': 'Health_Insur_PT',
    'CP1': 'Edu_Programs',
    'WL6': 'Work_From_Home_Allowed',
    'WD1_1':'Under_30',
    'WD1_2': '60_Plus',
    'WD2': 'Sex_F',
    'WD3': 'Hourly/Non-exempt',
    'WD4': 'No_Daytime_Shift',
    'WD5': 'Remote/Off-Site',
    'WD6': 'Unionized',
    'WD7': 'Avg_Annual_Turnover_Per'
}

In [9]:
df = df.rename(columns=newcols)

In [10]:
df.head().T

,0,1,2,3,4
Industry,7.0,7.0,7.0,7.0,7.0
Employee_Size,7.0,6.0,8.0,4.0,4.0
Organization_Type_Profit,3.0,3.0,3.0,2.0,3.0
Health_Insur_Type,2.0,2.0,1.0,1.0,1.0
Personal_Health_Insur,1.0,3.0,3.0,2.0,3.0
Health_Insur_PT,2.0,1.0,1.0,1.0,1.0
Edu_Programs,1.0,1.0,1.0,2.0,1.0
Work_From_Home_Allowed,1.0,1.0,1.0,2.0,1.0
Under_30,25.0,997.0,35.0,50.0,50.0
60_Plus,20.0,997.0,4.0,15.0,40.0


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [11]:
df["Industry"].isna().sum()

1

In [12]:
df["Industry"].fillna(0, inplace=True)

In [13]:
df["Industry"].isna().sum()

0

In [14]:
industry_codes = {"Industry":{
    1: "Manual Labor and Resources",
    2: "Trade, Transportion, Warehousing",
    3: "Arts and Customer Service",
    4: "Business and Finance",
    5: "Public Services",
    6: "Public Admin",
    7: "Hospital Worksites",
    0: "Unrecorded"
}}

In [15]:
df = df.replace(industry_codes)

In [16]:
df.head().T

,0,1,2,3,4
Industry,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites
Employee_Size,7.0,6.0,8.0,4.0,4.0
Organization_Type_Profit,3.0,3.0,3.0,2.0,3.0
Health_Insur_Type,2.0,2.0,1.0,1.0,1.0
Personal_Health_Insur,1.0,3.0,3.0,2.0,3.0
Health_Insur_PT,2.0,1.0,1.0,1.0,1.0
Edu_Programs,1.0,1.0,1.0,2.0,1.0
Work_From_Home_Allowed,1.0,1.0,1.0,2.0,1.0
Under_30,25.0,997.0,35.0,50.0,50.0
60_Plus,20.0,997.0,4.0,15.0,40.0


## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [17]:
df["Employee_Size"] = df["Employee_Size"].replace([[1,2,3], [4,5], [6,7,8]], ["Small", "Medium", "Large"])
df["Employee_Size"].value_counts()

Employee_Size
Small     2195
Medium     393
Large      254
Name: count, dtype: int64

In [18]:
df.head().T

,0,1,2,3,4
Industry,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites
Employee_Size,Large,Large,Large,Medium,Medium
Organization_Type_Profit,3.0,3.0,3.0,2.0,3.0
Health_Insur_Type,2.0,2.0,1.0,1.0,1.0
Personal_Health_Insur,1.0,3.0,3.0,2.0,3.0
Health_Insur_PT,2.0,1.0,1.0,1.0,1.0
Edu_Programs,1.0,1.0,1.0,2.0,1.0
Work_From_Home_Allowed,1.0,1.0,1.0,2.0,1.0
Under_30,25.0,997.0,35.0,50.0,50.0
60_Plus,20.0,997.0,4.0,15.0,40.0


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

**Why do this way when other way is faster**

code_book = {'Organization_Type_Profit': {1: 'For Profit, Public',
                      2: 'For Profit, Private',3: 'Non-Profit',
                      4: 'State or Local Govt',
                      5: 'Federal Govt',
                      6: 'Other',
                      97: np.nan,
                      98: np.nan,
                      99: np.nan},
            'Health_Insur_Type': {1: 'Full',
                                          2: 'Partial',
                                          3: 'No',
                                          97: np.nan,
                                          98: np.nan,
                                          99: np.nan},
            'Personal_Health_Insur': {1: 'Larger',
                                         2: 'Smaller',
                                         3: 'About Same',
                                         96: np.nan,
                                         97: np.nan,
                                         98: np.nan,
                                         99: np.nan},
            'Health_Insur_PT': {1: 'Yes',
                                           2: 'No',
                                           97: np.nan,
                                           98: np.nan,
                                           99: np.nan},
            'Edu_Programs': {1: 'Yes',
                                         2: 'No',
                                         97: np.nan,
                                         98: np.nan,
                                         99: np.nan},
            'Work_From_Home_Allowed': {1: 'Yes',
                    2: 'No',
                    97: np.nan,
                    98: np.nan,
                    99: np.nan}}

df = df.replace(code_book)

In [19]:
df["Organization_Type_Profit"] = df["Organization_Type_Profit"].replace([1,2,3,4,5,6, [97,98,99]],
                                                                       ["for profit-public", "for profit-private", "non-profit", 
                                                                        "state or local govt", "fed govt", "other", np.nan])
df["Organization_Type_Profit"].value_counts()

Organization_Type_Profit
for profit-private     1199
non-profit              632
state or local govt     378
for profit-public       317
other                    51
fed govt                 36
Name: count, dtype: int64

In [20]:
df["Organization_Type_Profit"].isna().sum()

230

In [21]:
df["Health_Insur_Type"] = df["Health_Insur_Type"].replace([1,2,3,[97,98,99]],
                                                          ["Full","Partial","No",np.nan])

In [22]:
df["Personal_Health_Insur"] = df["Personal_Health_Insur"].replace([1,2,3,[96, 97, 98, 99]], 
                                                                  ["Larger", "Smaller", "About the same", np.nan])

In [23]:
df["Health_Insur_PT"] = df["Health_Insur_PT"].replace([1,2,[97,98,99]],
                                                        ["yes","no",np.nan])

In [24]:
df["Edu_Programs"] = df["Edu_Programs"].replace([1,2,[97,98,99]],
                                                        ["yes","no",np.nan])

In [25]:
df["Work_From_Home_Allowed"] = df["Work_From_Home_Allowed"].replace([1,2,[97,98,99]],
                                                        ["yes","no",np.nan])

In [26]:
df.head().T

,0,1,2,3,4
Industry,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites
Employee_Size,Large,Large,Large,Medium,Medium
Organization_Type_Profit,non-profit,non-profit,non-profit,for profit-private,non-profit
Health_Insur_Type,Partial,Partial,Full,Full,Full
Personal_Health_Insur,Larger,About the same,About the same,Smaller,About the same
Health_Insur_PT,no,yes,yes,yes,yes
Edu_Programs,yes,yes,yes,no,yes
Work_From_Home_Allowed,yes,yes,yes,no,yes
Under_30,25.0,997.0,35.0,50.0,50.0
60_Plus,20.0,997.0,4.0,15.0,40.0


## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [27]:
df = df.replace([997,998,999],
                np.nan)

In [28]:
df.head().T

,0,1,2,3,4
Industry,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites,Hospital Worksites
Employee_Size,Large,Large,Large,Medium,Medium
Organization_Type_Profit,non-profit,non-profit,non-profit,for profit-private,non-profit
Health_Insur_Type,Partial,Partial,Full,Full,Full
Personal_Health_Insur,Larger,About the same,About the same,Smaller,About the same
Health_Insur_PT,no,yes,yes,yes,yes
Edu_Programs,yes,yes,yes,no,yes
Work_From_Home_Allowed,yes,yes,yes,no,yes
Under_30,25.0,NaN,35.0,50.0,50.0
60_Plus,20.0,NaN,4.0,15.0,40.0


In [29]:
df.isna().sum()

Industry                       0
Employee_Size                  1
Organization_Type_Profit     230
Health_Insur_Type             80
Personal_Health_Insur        594
Health_Insur_PT              161
Edu_Programs                 175
Work_From_Home_Allowed       294
Under_30                     969
60_Plus                      942
Sex_F                        796
Hourly/Non-exempt            936
No_Daytime_Shift             857
Remote/Off-Site              785
Unionized                    692
Avg_Annual_Turnover_Per     1286
dtype: int64

## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [32]:
df.sort_values(by=["Industry", "Employee_Size"], ascending=True).sort_values(by="Under_30", ascending=False)

,Industry,Employee_Size,Organization_Type_Profit,Health_Insur_Type,Personal_Health_Insur,Health_Insur_PT,Edu_Programs,Work_From_Home_Allowed,Under_30,60_Plus,Sex_F,Hourly/Non-exempt,No_Daytime_Shift,Remote/Off-Site,Unionized,Avg_Annual_Turnover_Per
554,"Trade, Transportion, Warehousing",Small,for profit-private,NaN,NaN,no,yes,yes,100.0,0.0,75.0,80.0,0.0,0.0,0.0,NaN
2398,Arts and Customer Service,Small,NaN,NaN,NaN,no,NaN,no,100.0,0.0,90.0,100.0,0.0,0.0,NaN,NaN
2034,Manual Labor and Resources,Small,for profit-private,NaN,NaN,no,no,no,100.0,0.0,40.0,95.0,100.0,0.0,0.0,NaN
2639,Arts and Customer Service,Small,for profit-private,No,NaN,no,no,no,100.0,0.0,40.0,100.0,100.0,0.0,0.0,10.0
900,Manual Labor and Resources,Small,NaN,No,NaN,no,no,yes,100.0,0.0,90.0,90.0,1.0,1.0,0.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,"Trade, Transportion, Warehousing",Small,non-profit,Full,About the same,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,"Trade, Transportion, Warehousing",Small,for profit-private,Partial,Larger,yes,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,"Trade, Transportion, Warehousing",Small,for profit-public,Full,Larger,no,yes,yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0
2631,"Trade, Transportion, Warehousing",Small,for profit-private,Partial,Larger,yes,no,no,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]